In [4]:
import numpy as np

class QLearningAgent:
    def __init__(self, n_states, n_actions, learning_rate=0.1, discount_factor=0.9, exploration_prob=0.2):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob
        self.n_states = n_states
        self.n_actions = n_actions
        self.q_table = np.zeros((n_states, n_actions))

    def choose_action(self, state, explore=True):
        if explore and np.random.rand() < self.exploration_prob:
            # Explore: choose a random action
            return np.random.choice(self.n_actions)
        else:
            # Exploit: choose the action with the highest Q value
            return np.argmax(self.q_table[state, :])

    def update_q_value(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state, :])
        td_target = reward + self.discount_factor * self.q_table[next_state, best_next_action]
        td_error = td_target - self.q_table[state, action]
        self.q_table[state, action] += self.learning_rate * td_error

def run_maze():
    # Define the maze as a 2D array
    maze = np.array([
        [0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 1, 1],
        [0, 1, 0, 0, 0]
    ])

    n_states = np.prod(maze.shape)
    n_actions = 4  # 0: up, 1: down, 2: left, 3: right

    agent = QLearningAgent(n_states, n_actions)

    # Convert 2D coordinates to a single index
    def state_to_index(state):
        return state[0] * maze.shape[1] + state[1]

    # Convert single index to 2D coordinates
    def index_to_state(index):
        return divmod(index, maze.shape[1])
    
    # Check if the action is valid (i.e., not hitting a wall)
    def check_action(action, state):
        if action == 0 and state[0] > 0 and maze[state[0] - 1, state[1]] != 1:
            return (state[0] - 1, state[1])
        if action == 1 and state[0] < maze.shape[0] - 1 and maze[state[0] + 1, state[1]] != 1:
            return (state[0] + 1, state[1])
        if action == 2 and state[1] > 0 and maze[state[0], state[1] - 1] != 1:
            return (state[0], state[1] - 1)
        if action == 3 and state[1] < maze.shape[1] - 1 and maze[state[0], state[1] + 1] != 1:
            return (state[0], state[1] + 1)
        # Stay in the current state if the action is invalid
        return state

    # Training loop
    epochs = 1000
    for epoch in range(epochs):
        state = (0, 0)  # Starting position
        goal = (4, 4)   # Goal position

        while state != goal:
            index = state_to_index(state)
            action = agent.choose_action(index)
            next_state = check_action(action, state)

            # Calculate reward (negative for each step, additional penalty for hitting walls)
            reward = -1 if state != goal else 0
            if state == next_state:
                reward -= 5  # Additional penalty for hitting walls

            # Update Q value
            agent.update_q_value(index, action, reward, state_to_index(next_state))

            # Move to the next state
            state = next_state

    # Testing
    state = (0, 0)
    path = [state]
    while state != goal:
        index = state_to_index(state)
        action = agent.choose_action(index, explore=False)  # Exploit only, no exploration
        next_state = check_action(action, state)

        path.append(next_state)
        state = next_state

    print("Optimal path:", path)

if __name__ == "__main__":
    run_maze()

Optimal path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
